# GluonTS - Transfer Learning Approach (quarterly) 

- Two datasets: A and B
- Dataset A: Dataset subsetted by frequency, i.e. `m4_quarterly_atm`
- Dataset B: Dataset subsetted by frequency and by domain, i.e. `m4_quarterly_industry`
- Network A: Trained on dataset A
- Network B: Trained on dataset B
- Both networks are evaluated on dataset B. 

- Adjust epochs to account for different size of the datasets

Goal: Check TZ1: "Does the network learn from time series of the same domain or is it helpful to use time series from other domains?"

- Both networks are trained for ten trials (deepar) or three trials (deepstate)

Note that there are already results where a network is trained on `m4_frequency_domain` and evaluated on the same network.

In [1]:
!pip install mxnet=="1.4.1"

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install gluonts

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Imports



In [3]:
import gluonts
import mxnet

print(gluonts.__version__)
print(mxnet.__version__)

0.4.2
1.4.1


In [4]:
# standard imports
import numpy as np
import pandas as pd

# Vis
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint

# mxnet
import mxnet as mx

# gluonts
from gluonts.dataset.util import to_pandas

# imports from utility function
import utils_gluonts10 as utils
from utils_gluonts10 import get_dataset, dataset_recipes
from utils_gluonts10_deepar import deepar


INFO:root:Using CPU


In [5]:
print(f"Available datasets: {list(dataset_recipes.keys())}")

Available datasets: ['m4_daily', 'm4_daily_domain', 'm4_daily_id', 'm4_hourly', 'm4_hourly_id', 'm4_monthly', 'm4_monthly_atm', 'm4_monthly_dates', 'm4_monthly_demographic', 'm4_monthly_domain', 'm4_monthly_end032014', 'm4_monthly_end052015', 'm4_monthly_end092007', 'm4_monthly_finance', 'm4_monthly_id', 'm4_monthly_industry', 'm4_monthly_macro', 'm4_monthly_micro', 'm4_monthly_micro_atm', 'm4_monthly_nonmicro_10975_seed42', 'm4_monthly_nonmicro_10975_seed43', 'm4_monthly_nonmicro_10975_seed44', 'm4_monthly_nonmicro_10975_seed45', 'm4_monthly_nonmicro_10975_seed46', 'm4_monthly_nonmicro_10975_seed47', 'm4_monthly_nonmicro_10975_seed48', 'm4_monthly_nonmicro_10975_seed49', 'm4_monthly_nonmicro_10975_seed50', 'm4_monthly_nonmicro_10975_seed51', 'm4_monthly_other', 'm4_quarterly', 'm4_quarterly_atm', 'm4_quarterly_dates', 'm4_quarterly_demographic', 'm4_quarterly_domain', 'm4_quarterly_end2003q3', 'm4_quarterly_end2013q1', 'm4_quarterly_end2015q1', 'm4_quarterly_finance', 'm4_quarterly_id

In [6]:
dataset = get_dataset("m4_weekly", regenerate=False)

INFO:root:downloading and processing m4_weekly


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


### mv the data

Before moving on: Move the respective data according the required folder structure. In AWS:

- `mv [dataname] ../../.mxnet/gluon-ts/datasets/`

## Check the data

In [7]:
dataname_a = "m4_quarterly_atm"
dataset_a = get_dataset(dataname_a, regenerate=False)
len(dataset_a.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_atm.


24000

In [8]:
dataname_b1 = "m4_quarterly_macro"
dataset_b1 = get_dataset(dataname_b1, regenerate=False)
len(dataset_b1.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_macro.


5315

In [9]:
dataname_b2 = "m4_quarterly_micro"
dataset_b2 = get_dataset(dataname_b2, regenerate=False)
len(dataset_b2.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_micro.


6020

In [10]:
dataname_b3 = "m4_quarterly_other"
dataset_b3 = get_dataset(dataname_b3, regenerate=False)
len(dataset_b3.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_other.


865

In [11]:
dataname_b4 = "m4_quarterly_finance"
dataset_b4 = get_dataset(dataname_b4, regenerate=False)
len(dataset_b4.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_finance.


5305

In [12]:
dataname_b5 = "m4_quarterly_industry"
dataset_b5 = get_dataset(dataname_b5, regenerate=False)
len(dataset_b5.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.


4637

In [7]:
dataname_b6 = "m4_quarterly_demographic"
dataset_b6 = get_dataset(dataname_b6, regenerate=False)
len(dataset_b6.train)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.


1858

### Dataset A


In [8]:
train_entry_a = next(iter(dataset_a.train))
test_entry_a = next(iter(dataset_a.test))

print(train_entry_a.keys())
print(test_entry_a.keys())
print("length train:", len(dataset_a.train))
print("length test:", len(dataset_a.test))


print("\nstart:\n",train_entry_a["start"])
print("target:\n", train_entry_a["target"][:10])
print("feat_static_cat:\n", train_entry_a["feat_static_cat"])

train_series_a = to_pandas(train_entry_a)
test_series_a = to_pandas(test_entry_a)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series_a.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series_a.plot(ax=ax[1])
ax[1].axvline(train_series_a.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series", "end of train series"], loc="upper left")

plt.show()

NameError: name 'dataset_a' is not defined

## Dataset B

In [9]:
dataset_b = dataset_b1

train_entry_b = next(iter(dataset_b.train))
test_entry_b = next(iter(dataset_b.test))

print(train_entry_b.keys())
print(test_entry_b.keys())
print("length train:", len(dataset_b.train))
print("length test:", len(dataset_b.test))

print("\nstart:\n",train_entry_b["start"])
print("target:\n", train_entry_b["target"][:10])
print("feat_static_cat:\n", train_entry_b["feat_static_cat"])

train_series_b = to_pandas(train_entry_b)
test_series_b = to_pandas(test_entry_b)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series_b.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series_b.plot(ax=ax[1])
ax[1].axvline(train_series_b.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series", "end of train series"], loc="upper left")

plt.show()

NameError: name 'dataset_b1' is not defined

## Comparison of both datasets

- Compare whether both datasets have the same length


In [10]:
# Check length
print("Length test train:", len(dataset_a.train) == len(dataset_b.train))
print("Length test test:", len(dataset_a.test) == len(dataset_b.test))

# Check frequency
print("Frequency test:",(dataset_a.metadata.freq == dataset_b.metadata.freq))


NameError: name 'dataset_a' is not defined

## Modeling: DeepAR

In [11]:
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

from gluonts.model.deepar import DeepAREstimator

In [12]:
# A: dataset_a
# B: dataset_b
print(dataset_a)
print("\n", dataset_b)

NameError: name 'dataset_a' is not defined

In [13]:
print(dataname_a)
print(dataname_b1)

NameError: name 'dataname_a' is not defined

In [14]:
def deepar_train_eval(dataname_a, dataname_b, seed=42, epochs=100, num_batches=100):
    """
    Function that trains a network using data from dataset B and evaluates the trained network on dataset A.
    Dataset A is the dataset of interest for our analysis. 
    """    
    
    dataset_a = get_dataset(dataname_a, regenerate=False)
    dataset_b = get_dataset(dataname_b, regenerate=False)
    
    # Assert checks
    assert (dataset_a.metadata.freq == dataset_b.metadata.freq)
#     assert ( len(dataset_a.train)==len(dataset_b.train) )
#     assert ( len(dataset_a.test)==len(dataset_b.test))
    
    mx.random.seed(seed)
    np.random.seed(seed)

    # define trainer
    trainer=Trainer(
        ctx=mx.cpu(),
        epochs=epochs,
        num_batches_per_epoch=num_batches,
        learning_rate=1e-3,
    )

    estimator=DeepAREstimator(
        trainer=trainer,
        context_length=dataset_a.metadata.prediction_length,
        prediction_length=dataset_a.metadata.prediction_length,
        freq=dataset_a.metadata.freq,
        use_feat_static_cat=False,
    )
    
#     estimator = DeepAREstimator(
#         trainer=trainer,
#         cardinality= [
#             feat_static_cat.cardinality 
#             for feat_static_cat in dataset_a.metadata.feat_static_cat
#         ],
#         context_length=dataset_a.metadata.prediction_length,
#         prediction_length=dataset_a.metadata.prediction_length,
#         freq=dataset_a.metadata.freq,
#         use_feat_static_cat=True
#     )
    
    # predictor trained on dataset_a, evaluated on dataset_b
    predictor_a=estimator.train(training_data=dataset_a.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset_b.test, predictor=predictor_a, num_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series = len(dataset_b.test))
    
    metrics=["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    
    output["seed"]=seed
    output["epochs"]=epochs
    output["num_batches"]=num_batches
    output["train data"]=dataname_a
    output["eval data"]=dataname_b
    
    df=pd.DataFrame([output])

    return df

    

## Subsets

In [15]:
print(dataname_a)
print(dataname_b1)

NameError: name 'dataname_a' is not defined

## Study 1: `m4_quarterly_atm` vs. `m4_quarterly_macro` trained network

- Alternating between Run I - 10 trials A:B vs. 
- Run II - 10 trials of B:B


In [1]:
# results = pd.DataFrame()

# for i in range(42,52):
#     print("Next seed:", i)
#     df = deepar_train_eval(dataname_a="m4_quarterly_macro", dataname_b="m4_quarterly_macro", seed=i, epochs=100, num_batches=50)
#     pprint(df)
#     results = results.append(df)
    
# results

## Study 2: `m4_quarterly_atm` vs. `m4_quarterly_micro` trained network

- Alternating between A:B and B:B

In [2]:
# results = pd.DataFrame()

# for i in range(42,52):
#     print("Next seed:", i)
#     df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_micro", seed=i, epochs=100, num_batches=50)
#     pprint(df)
#     results = results.append(df)
    
# results

## Study 3: `m4_quarterly_atm` vs `m4_quarterly_other`

- Alternating between A:B and B:B

In [ ]:
# results = pd.DataFrame()

# for i in range(42,45):
#     print("Next seed:", i)
#     df = deepar_train_eval(dataname_a="m4_quarterly_other", dataname_b="m4_quarterly_other", seed=i, epochs=400, num_batches=200)
#     pprint(df)
#     results = results.append(df)
    
# results

## Study 4: `m4_quarterly_atm` vs `m4_quarterly_finance`

In [ ]:
# results = pd.DataFrame()

# for i in range(42,52):
#     print("Next seed:", i)
#     df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_finance", seed=i, epochs=100, num_batches=50)
#     pprint(df)
#     results = results.append(df)
    
# results

In [ ]:
results = pd.DataFrame()

for i in range(52,62):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_finance", dataname_b="m4_quarterly_finance", seed=i, epochs=100, num_batches=50)
    pprint(df)
    results = results.append(df)
    
results

In [ ]:
results = pd.DataFrame()

for i in range(45,47):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_finance", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

In [ ]:
results = pd.DataFrame()

for i in range(45,47):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_finance", dataname_b="m4_quarterly_finance", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

## Study 5: `m4_quarterly_atm` vs. `m4_quarterly_industry`

In [ ]:
# results = pd.DataFrame()

# for i in range(42,52):
#     print("Next seed:", i)
#     df = deepar_train_eval(dataname_a="m4_quarterly_industry", dataname_b="m4_quarterly_industry", seed=i, epochs=100, num_batches=50)
#     pprint(df)
#     results = results.append(df)
    
# results

In [31]:
results = pd.DataFrame()

for i in range(42,52):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_industry", dataname_b="m4_quarterly_industry", seed=i, epochs=200, num_batches=100)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 100/100 [00:01<00:00, 70.36it/s, avg_epoch_loss=7.91]
INFO:root:Epoch[0] Elapsed time 1.423 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.914683
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.96it/s, avg_epoch_loss=7.43]
INFO:root:Epoch[1] Elapsed time 1.391 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.432396
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.76it/s, avg_epoch_loss=7.41]
INFO:root:Epoch[2] Elapsed time 1.395 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.412414
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 72.58it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[3] Elapsed time 1.379 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.212453
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.07it/s, avg_epoch_loss=7.25]
INFO:root:Epoch[4] Elapsed time 1.429 seconds
INFO:root:E

INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 71.05it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[74] Elapsed time 1.409 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.714274
INFO:root:Loading parameters from best epoch (64)
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 68.61it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[75] Elapsed time 1.459 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.795312
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 70.66it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[76] Elapsed time 1.417 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.827966
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 72.14it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[77] Elapsed time 1.388 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.732759
INFO:root:Epoch[78] Learning rate is 0.00025
100%|██████████

INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=6.682716
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.30it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[110] Elapsed time 1.424 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.779538
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.54it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[111] Elapsed time 1.419 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.775925
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 72.63it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[112] Elapsed time 1.378 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.727176
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.77it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[113] Elapsed time 1.414 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.853198
INFO:root:Epoch[114] Learning rate is 5e-05

...

INFO:root:Epoch[110] Learning rate is 0.000125
100%|██████████| 100/100 [00:01<00:00, 70.88it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[110] Elapsed time 1.412 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.653148
INFO:root:Loading parameters from best epoch (100)
INFO:root:Epoch[111] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:01<00:00, 69.35it/s, avg_epoch_loss=6.75]
INFO:root:Epoch[111] Elapsed time 1.443 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.749922
INFO:root:Epoch[112] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:01<00:00, 71.74it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[112] Elapsed time 1.395 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.821734
INFO:root:Epoch[113] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:01<00:00, 72.82it/s, avg_epoch_loss=6.69]
INFO:root:Epoch[113] Elapsed time 1.375 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.690797
INFO:root:Epoch[114] Learning rate is 6.2

INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.49it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[146] Elapsed time 1.440 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=6.798147
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.42it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[147] Elapsed time 1.421 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.744508
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.77it/s, avg_epoch_loss=6.76]
INFO:root:Epoch[148] Elapsed time 1.395 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=6.761074
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.18it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[149] Elapsed time 1.426 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=6.726674
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.17it/s, avg_epoch_l

INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=6.677839
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.38it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[183] Elapsed time 1.402 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.814310
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.99it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[184] Elapsed time 1.390 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.647046
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.51it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[185] Elapsed time 1.440 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=6.734320
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.97it/s, avg_epoch_loss=6.75]
INFO:root:Epoch[186] Elapsed time 1.391 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=6.745903
INFO:root:Epoch[187] Learning rate is 5e-05

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.182478  0.094444  14.749207            0.085232            0.053014   

   seed  epochs  num_batches             train data              eval data  
0    51     200          100  m4_quarterly_industry  m4_quarterly_industry  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.192101,0.095404,14.704793,0.085587,0.053698,42,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.217150,0.095416,14.847556,0.085633,0.053191,43,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.201341,0.096297,15.169504,0.086693,0.055098,44,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.173009,0.093865,14.711837,0.084551,0.052557,45,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.227503,0.096991,14.741935,0.086410,0.053972,46,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.170302,0.094531,13.527344,0.084711,0.049945,47,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.215917,0.096628,15.342017,0.086716,0.053606,48,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.214718,0.096364,15.399178,0.086970,0.055078,49,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.194645,0.095668,15.449274,0.086261,0.055160,50,200,100,m4_quarterly_industry,m4_quarterly_industry
0,1.182478,0.094444,14.749207,0.085232,0.053014,51,200,100,m4_quarterly_industry,m4_quarterly_industry


In [32]:
results = pd.DataFrame()

for i in range(42,45):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_industry", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_atm.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 200/200 [00:02<00:00, 70.76it/s, avg_epoch_loss=7.69]
INFO:root:Epoch[0] Elapsed time 2.828 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.687089
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 69.63it/s, avg_epoch_loss=7.18]
INFO:root:Epoch[1] Elapsed time 2.874 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.180051
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 71.05it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[2] Elapsed time 2.816 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.193969
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 69.45it/s, avg_epoch_loss=7.1]
INFO:root:Epoch[3] Elapsed time 2.881 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.104381
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 71.47it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[4] Elapsed time 2.800 seconds
INFO:root:Ep

INFO:root:Epoch[37] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 71.05it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[37] Elapsed time 2.816 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.827728
INFO:root:Epoch[38] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.93it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[38] Elapsed time 2.821 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.792642
INFO:root:Epoch[39] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.07it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[39] Elapsed time 2.855 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.802193
INFO:root:Epoch[40] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.58it/s, avg_epoch_loss=6.89]
INFO:root:Epoch[40] Elapsed time 2.835 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.888149
INFO:root:Epoch[41] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.43it/s, avg_epoch_loss

INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.36it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[110] Elapsed time 2.885 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.781739
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.22it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[111] Elapsed time 2.850 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.792256
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.28it/s, avg_epoch_loss=6.76]
INFO:root:Epoch[112] Elapsed time 2.888 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.756591
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.87it/s, avg_epoch_loss=6.67]
INFO:root:Epoch[113] Elapsed time 2.823 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.674271
INFO:root:Epoch[114] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.71it/s, avg_epoch_

...

INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=6.432178
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.10it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[220] Elapsed time 2.855 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=6.491991
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.21it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[221] Elapsed time 2.850 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=6.452930
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.34it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[222] Elapsed time 2.845 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=6.335701
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.74it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[223] Elapsed time 2.829 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=6.459334
INFO:root:Epoch[224] Learning rate is 5e-05

INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.50it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[293] Elapsed time 2.839 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.307912
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.31it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[294] Elapsed time 2.806 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.421142
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.18it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[295] Elapsed time 2.811 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.470115
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.12it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[296] Elapsed time 2.854 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.417148
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.46it/s, avg_epoch_

INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=6.490694
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.39it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[330] Elapsed time 2.803 seconds
INFO:root:Epoch[330] Evaluation metric 'epoch_loss'=6.366264
INFO:root:Epoch[331] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.19it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[331] Elapsed time 2.810 seconds
INFO:root:Epoch[331] Evaluation metric 'epoch_loss'=6.442432
INFO:root:Epoch[332] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 64.95it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[332] Elapsed time 3.081 seconds
INFO:root:Epoch[332] Evaluation metric 'epoch_loss'=6.395636
INFO:root:Epoch[333] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.90it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[333] Elapsed time 2.823 seconds
INFO:root:Epoch[333] Evaluation metric 'epoch_loss'=6.392166
INFO:root:Epoch[334] Learning rate is 5e-05


       MASE     sMAPE      MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  seed  \
0  1.200472  0.094649  11.41639            0.085309            0.046786    44   

   epochs  num_batches        train data              eval data  
0     400          200  m4_quarterly_atm  m4_quarterly_industry  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.192224,0.095214,10.427667,0.085654,0.050181,42,400,200,m4_quarterly_atm,m4_quarterly_industry
0,1.258656,0.097659,12.747016,0.087693,0.050618,43,400,200,m4_quarterly_atm,m4_quarterly_industry
0,1.200472,0.094649,11.416390,0.085309,0.046786,44,400,200,m4_quarterly_atm,m4_quarterly_industry


In [33]:
results = pd.DataFrame()

for i in range(42,45):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_industry", dataname_b="m4_quarterly_industry", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_industry.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 200/200 [00:02<00:00, 69.17it/s, avg_epoch_loss=7.67]
INFO:root:Epoch[0] Elapsed time 2.893 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.673539
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.48it/s, avg_epoch_loss=7.31]
INFO:root:Epoch[1] Elapsed time 2.839 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.312433
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 69.47it/s, avg_epoch_loss=7.25]
INFO:root:Epoch[2] Elapsed time 2.880 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.254974
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 71.31it/s, avg_epoch_loss=7.08]
INFO:root:Epoch[3] Elapsed time 2.806 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.075004
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 71.12it/s, avg_epoch_loss=7.07]
INFO:root:Epoch[4] Elapsed time 2.813 seconds
INFO:root:E

100%|██████████| 200/200 [00:02<00:00, 69.92it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[74] Elapsed time 2.862 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.495492
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.81it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[75] Elapsed time 2.826 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.515733
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 69.60it/s, avg_epoch_loss=6.56]
INFO:root:Epoch[76] Elapsed time 2.875 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.559277
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.61it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[77] Elapsed time 2.834 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.472345
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.78it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[78] Elapsed time 2.827 seconds

INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.442075
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 69.94it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[111] Elapsed time 2.861 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.474506
INFO:root:Epoch[112] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.89it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[112] Elapsed time 2.823 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.441751
INFO:root:Epoch[113] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.64it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[113] Elapsed time 2.832 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.490236
INFO:root:Epoch[114] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.78it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[114] Elapsed time 2.827 seconds
INFO:root:Epoch[114] Evaluation metric 'epoch_loss'=6.474327
INFO:root:Epoch[115] Learning rate 

...

INFO:root:Epoch[256] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.56it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[256] Elapsed time 2.877 seconds
INFO:root:Epoch[256] Evaluation metric 'epoch_loss'=6.396626
INFO:root:Epoch[257] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.04it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[257] Elapsed time 2.898 seconds
INFO:root:Epoch[257] Evaluation metric 'epoch_loss'=6.390051
INFO:root:Epoch[258] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.47it/s, avg_epoch_loss=6.36]
INFO:root:Epoch[258] Elapsed time 2.880 seconds
INFO:root:Epoch[258] Evaluation metric 'epoch_loss'=6.363752
INFO:root:Epoch[259] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.23it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[259] Elapsed time 2.849 seconds
INFO:root:Epoch[259] Evaluation metric 'epoch_loss'=6.369115
INFO:root:Epoch[260] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 68.96it/s, avg_epoch_l

INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=6.370088
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 72.03it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[293] Elapsed time 2.778 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.404327
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.27it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[294] Elapsed time 2.848 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.425272
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.60it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[295] Elapsed time 2.834 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.311547
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.48it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[296] Elapsed time 2.800 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.369614
INFO:root:Epoch[297] Learning rate is 5e-05


INFO:root:Epoch[366] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.07it/s, avg_epoch_loss=6.32]
INFO:root:Epoch[366] Elapsed time 2.856 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=6.316907
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.97it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[367] Elapsed time 2.780 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=6.387054
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.55it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[368] Elapsed time 2.837 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=6.385560
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.12it/s, avg_epoch_loss=6.36]
INFO:root:Epoch[369] Elapsed time 2.854 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=6.362109
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.28it/s, avg_epoch_

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.140343  0.092134  11.094694            0.082887            0.045229   

   seed  epochs  num_batches             train data              eval data  
0    44     400          200  m4_quarterly_industry  m4_quarterly_industry  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.145256,0.092354,11.719948,0.082988,0.045883,42,400,200,m4_quarterly_industry,m4_quarterly_industry
0,1.146002,0.092777,11.399090,0.083439,0.045424,43,400,200,m4_quarterly_industry,m4_quarterly_industry
0,1.140343,0.092134,11.094694,0.082887,0.045229,44,400,200,m4_quarterly_industry,m4_quarterly_industry


## Study 6: `m4_quarterly_atm` vs. `m4_quarterly_demographic`

In [34]:
results = pd.DataFrame()

for i in range(42,52):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_demographic", seed=i, epochs=100, num_batches=50)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_atm.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/50 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 50/50 [00:00<00:00, 65.48it/s, avg_epoch_loss=8.26]
INFO:root:Epoch[0] Elapsed time 0.765 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.263358
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 73.78it/s, avg_epoch_loss=7.63]
INFO:root:Epoch[1] Elapsed time 0.679 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.633776
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 73.40it/s, avg_epoch_loss=7.59]
INFO:root:Epoch[2] Elapsed time 0.683 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.591601
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 68.86it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[3] Elapsed time 0.727 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.259622
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 66.46it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[4] Elapsed time 0.754 seconds
INFO:root:Epoch[4] Ev

INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:00<00:00, 69.75it/s, avg_epoch_loss=7.1]
INFO:root:Epoch[74] Elapsed time 0.719 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=7.102310
INFO:root:Loading parameters from best epoch (64)
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 68.11it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[75] Elapsed time 0.735 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.797378
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 68.33it/s, avg_epoch_loss=7.08]
INFO:root:Epoch[76] Elapsed time 0.733 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=7.075775
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 72.32it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[77] Elapsed time 0.693 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=7.207502
INFO:root:Epoch[78] Learning rate is 6.25e-05
100%|██████████| 5

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.449014  0.109635  21.719362            0.121767            0.078508   

   seed  epochs  num_batches        train data                 eval data  
0    42     100           50  m4_quarterly_atm  m4_quarterly_demographic  
Next seed: 43


...

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.462082  0.109741  21.992186            0.119848            0.081663   

   seed  epochs  num_batches        train data                 eval data  
0    50     100           50  m4_quarterly_atm  m4_quarterly_demographic  
Next seed: 51


100%|██████████| 50/50 [00:00<00:00, 65.93it/s, avg_epoch_loss=8.18]
INFO:root:Epoch[0] Elapsed time 0.760 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.178529
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 75.40it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[1] Elapsed time 0.664 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.682076
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 73.12it/s, avg_epoch_loss=7.69]
INFO:root:Epoch[2] Elapsed time 0.685 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.686630
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.60it/s, avg_epoch_loss=7.35]
INFO:root:Epoch[3] Elapsed time 0.720 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.350398
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 67.33it/s, avg_epoch_loss=7.14]
INFO:root:Epoch[4] Elapsed time 0.744 seconds
INFO:root:Epoch[4] Ev

INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.980415
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 70.96it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[38] Elapsed time 0.706 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.896650
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 71.58it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[39] Elapsed time 0.700 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.920041
INFO:root:Loading parameters from best epoch (19)
INFO:root:Epoch[40] Learning rate is 0.00025
100%|██████████| 50/50 [00:00<00:00, 70.60it/s, avg_epoch_loss=7.01]
INFO:root:Epoch[40] Elapsed time 0.709 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=7.010159
INFO:root:Epoch[41] Learning rate is 0.00025
100%|██████████| 50/50 [00:00<00:00, 68.69it/s, avg_epoch_loss=7.22]
INFO:root:Epoch[41] Elapsed time 0.729 seconds
INFO:root:Epoch[41] Evaluation metric 'epoch_loss'=7.216812
INFO:root

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.424473  0.109935  20.022496            0.121039            0.078053   

   seed  epochs  num_batches        train data                 eval data  
0    51     100           50  m4_quarterly_atm  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.449014,0.109635,21.719362,0.121767,0.078508,42,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.657819,0.115306,20.778595,0.125616,0.075674,43,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.597210,0.114355,20.603788,0.124123,0.075656,44,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.544384,0.110906,23.281450,0.120130,0.083966,45,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.587113,0.112694,20.134263,0.123059,0.077236,46,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.423423,0.110081,20.679212,0.120918,0.079714,47,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,2.271942,0.132101,23.287406,0.137380,0.074536,48,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.505286,0.111001,20.231886,0.120657,0.079760,49,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.462082,0.109741,21.992186,0.119848,0.081663,50,100,50,m4_quarterly_atm,m4_quarterly_demographic
0,1.424473,0.109935,20.022496,0.121039,0.078053,51,100,50,m4_quarterly_atm,m4_quarterly_demographic


In [35]:
results = pd.DataFrame()

for i in range(42,52):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_demographic", dataname_b="m4_quarterly_demographic", seed=i, epochs=100, num_batches=50)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/50 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 50/50 [00:00<00:00, 68.14it/s, avg_epoch_loss=8.24]
INFO:root:Epoch[0] Elapsed time 0.735 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.240706
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 68.35it/s, avg_epoch_loss=7.55]
INFO:root:Epoch[1] Elapsed time 0.733 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.546876
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.68it/s, avg_epoch_loss=7.41]
INFO:root:Epoch[2] Elapsed time 0.719 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.413696
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.71it/s, avg_epoch_loss=7.42]
INFO:root:Epoch[3] Elapsed time 0.708 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.423411
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.49it/s, avg_epoch_loss=7.32]
INFO:root:Epoch[4] Elapsed time 0.711 seconds
INFO:root:Epoch[4] Ev

INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.47it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[38] Elapsed time 0.711 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.965037
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.31it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[39] Elapsed time 0.712 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.975985
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 71.10it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[40] Elapsed time 0.704 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.942961
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.96it/s, avg_epoch_loss=7.08]
INFO:root:Epoch[41] Elapsed time 0.706 seconds
INFO:root:Epoch[41] Evaluation metric 'epoch_loss'=7.084098
INFO:root:Epoch[42] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 72.61it/s, avg_epoch_loss=7.09]
INFO:root:Ep

100%|██████████| 50/50 [00:00<00:00, 71.37it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[75] Elapsed time 0.702 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.917406
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 70.62it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[76] Elapsed time 0.709 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.969991
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 72.37it/s, avg_epoch_loss=7]
INFO:root:Epoch[77] Elapsed time 0.692 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=7.001101
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 71.12it/s, avg_epoch_loss=7]
INFO:root:Epoch[78] Elapsed time 0.704 seconds
INFO:root:Epoch[78] Evaluation metric 'epoch_loss'=7.003960
INFO:root:Epoch[79] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<00:00, 72.22it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[79] Elapsed time 0.694 seconds
INFO:root:

...

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.443259  0.109983  19.539146            0.122123            0.077426   

   seed  epochs  num_batches                train data  \
0    50     100           50  m4_quarterly_demographic   

                  eval data  
0  m4_quarterly_demographic  
Next seed: 51


100%|██████████| 50/50 [00:00<00:00, 66.45it/s, avg_epoch_loss=8.28]
INFO:root:Epoch[0] Elapsed time 0.754 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.278403
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 68.62it/s, avg_epoch_loss=7.58]
INFO:root:Epoch[1] Elapsed time 0.730 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.579335
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 70.72it/s, avg_epoch_loss=7.54]
INFO:root:Epoch[2] Elapsed time 0.708 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.543003
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.21it/s, avg_epoch_loss=7.44]
INFO:root:Epoch[3] Elapsed time 0.724 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.435800
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.26it/s, avg_epoch_loss=7.42]
INFO:root:Epoch[4] Elapsed time 0.723 seconds
INFO:root:Epoch[4] Ev

INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 68.20it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[38] Elapsed time 0.735 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.982952
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.15it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[39] Elapsed time 0.724 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.988368
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 68.40it/s, avg_epoch_loss=6.96]
INFO:root:Epoch[40] Elapsed time 0.732 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.960796
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 69.63it/s, avg_epoch_loss=7.05]
INFO:root:Epoch[41] Elapsed time 0.720 seconds
INFO:root:Epoch[41] Evaluation metric 'epoch_loss'=7.054977
INFO:root:Loading parameters from best epoch (31)
INFO:root:Epoch[42] Learning rate is 0.0005
100%|██████████| 50/50 [00:00<

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.584133  0.112895  17.937302            0.121224            0.075581   

   seed  epochs  num_batches                train data  \
0    51     100           50  m4_quarterly_demographic   

                  eval data  
0  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.446678,0.111202,17.393811,0.121951,0.074632,42,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.458198,0.111547,20.292903,0.124752,0.078718,43,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.434993,0.111254,20.358403,0.124376,0.078321,44,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.452524,0.110638,18.671166,0.122099,0.076079,45,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.599799,0.113381,17.345327,0.122032,0.073274,46,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.517378,0.112296,18.694216,0.122973,0.075121,47,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.485670,0.111686,19.231499,0.123823,0.077786,48,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.733176,0.116381,18.549618,0.126284,0.076163,49,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.443259,0.109983,19.539146,0.122123,0.077426,50,100,50,m4_quarterly_demographic,m4_quarterly_demographic
0,1.584133,0.112895,17.937302,0.121224,0.075581,51,100,50,m4_quarterly_demographic,m4_quarterly_demographic


In [36]:
results = pd.DataFrame()

for i in range(42,52):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_demographic", seed=i, epochs=200, num_batches=100)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_atm.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 100/100 [00:01<00:00, 70.64it/s, avg_epoch_loss=7.95]
INFO:root:Epoch[0] Elapsed time 1.417 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.948567
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.76it/s, avg_epoch_loss=7.43]
INFO:root:Epoch[1] Elapsed time 1.395 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.425612
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 69.32it/s, avg_epoch_loss=7.13]
INFO:root:Epoch[2] Elapsed time 1.444 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.132203
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.14it/s, avg_epoch_loss=7.23]
INFO:root:Epoch[3] Elapsed time 1.407 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.227898
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 72.02it/s, avg_epoch_loss=7.09]
INFO:root:Epoch[4] Elapsed time 1.390 seconds
INFO:root:E

100%|██████████| 100/100 [00:01<00:00, 69.17it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[37] Elapsed time 1.447 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.827149
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 70.03it/s, avg_epoch_loss=7]
INFO:root:Epoch[38] Elapsed time 1.431 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=7.003809
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 69.36it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[39] Elapsed time 1.444 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.548514
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 69.51it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[40] Elapsed time 1.440 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.901964
INFO:root:Epoch[41] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 69.20it/s, avg_epoch_loss=6.88]
INFO:root:Epoch[41] Elapsed time 1.446 second

100%|██████████| 100/100 [00:01<00:00, 71.28it/s, avg_epoch_loss=6.88]
INFO:root:Epoch[109] Elapsed time 1.405 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=6.882766
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.22it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[110] Elapsed time 1.405 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=7.026332
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.89it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[111] Elapsed time 1.412 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.739722
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.41it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[112] Elapsed time 1.402 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.767785
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.13it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[113] Elapsed time

...

100%|██████████| 100/100 [00:01<00:00, 69.43it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[110] Elapsed time 1.442 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=7.032220
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.13it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[111] Elapsed time 1.427 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.984798
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 67.06it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[112] Elapsed time 1.493 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.782345
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 72.49it/s, avg_epoch_loss=7.02]
INFO:root:Epoch[113] Elapsed time 1.381 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=7.023472
INFO:root:Epoch[114] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 68.85it/s, avg_epoch_loss=6.56]
INFO:root:Epoch[114] Elapsed time

INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 72.73it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[147] Elapsed time 1.376 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.807323
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.59it/s, avg_epoch_loss=7.04]
INFO:root:Epoch[148] Elapsed time 1.439 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=7.044490
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 72.08it/s, avg_epoch_loss=6.91]
INFO:root:Epoch[149] Elapsed time 1.388 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=6.910826
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.70it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[150] Elapsed time 1.436 seconds
INFO:root:Epoch[150] Evaluation metric 'epoch_loss'=6.812506
INFO:root:Epoch[151] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 73.17it/s, avg_epoch_

INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.830527
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.58it/s, avg_epoch_loss=6.84]
INFO:root:Epoch[184] Elapsed time 1.420 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.838739
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.14it/s, avg_epoch_loss=7.01]
INFO:root:Epoch[185] Elapsed time 1.448 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=7.009788
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.71it/s, avg_epoch_loss=6.72]
INFO:root:Epoch[186] Elapsed time 1.416 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=6.723043
INFO:root:Epoch[187] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 68.91it/s, avg_epoch_loss=6.84]
INFO:root:Epoch[187] Elapsed time 1.453 seconds
INFO:root:Epoch[187] Evaluation metric 'epoch_loss'=6.838533
INFO:root:Epoch[188] Learning rate is 5e-05

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.541702  0.113776  20.651569            0.123067            0.075749   

   seed  epochs  num_batches        train data                 eval data  
0    51     200          100  m4_quarterly_atm  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.542184,0.115972,17.704293,0.122283,0.071923,42,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.408408,0.111436,19.931038,0.120299,0.075459,43,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.495873,0.115277,20.366747,0.124566,0.076722,44,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.469738,0.110940,21.199873,0.119772,0.079930,45,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.695786,0.120812,20.187759,0.128192,0.073627,46,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.527200,0.112253,22.064983,0.122106,0.079579,47,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.633100,0.116929,19.635772,0.127789,0.074968,48,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.531764,0.114599,20.284530,0.123707,0.075775,49,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.621142,0.113754,21.400890,0.123241,0.076918,50,200,100,m4_quarterly_atm,m4_quarterly_demographic
0,1.541702,0.113776,20.651569,0.123067,0.075749,51,200,100,m4_quarterly_atm,m4_quarterly_demographic


In [37]:
results = pd.DataFrame()

for i in range(42,52):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_demographic", dataname_b="m4_quarterly_demographic", seed=i, epochs=200, num_batches=100)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 100/100 [00:01<00:00, 68.50it/s, avg_epoch_loss=7.89]
INFO:root:Epoch[0] Elapsed time 1.461 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.893791
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.79it/s, avg_epoch_loss=7.42]
INFO:root:Epoch[1] Elapsed time 1.414 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.418554
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 67.88it/s, avg_epoch_loss=7.34]
INFO:root:Epoch[2] Elapsed time 1.475 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.344588
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.04it/s, avg_epoch_loss=7.29]
INFO:root:Epoch[3] Elapsed time 1.413 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.294632
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.88it/s, avg_epoch_loss=7.22]
INFO:root:Epoch[4] Elapsed time 1.412 seconds
INFO:root:E

100%|██████████| 100/100 [00:01<00:00, 72.37it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[74] Elapsed time 1.383 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.654175
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.70it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[75] Elapsed time 1.416 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.777918
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.44it/s, avg_epoch_loss=6.68]
INFO:root:Epoch[76] Elapsed time 1.421 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.676019
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.64it/s, avg_epoch_loss=6.64]
INFO:root:Epoch[77] Elapsed time 1.417 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.637088
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.28it/s, avg_epoch_loss=6.76]
INFO:root:Epoch[78] Elapsed time 1.404 second

INFO:root:Epoch[111] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 71.23it/s, avg_epoch_loss=6.73]
INFO:root:Epoch[111] Elapsed time 1.405 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.726886
INFO:root:Epoch[112] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 70.64it/s, avg_epoch_loss=6.63]
INFO:root:Epoch[112] Elapsed time 1.417 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.629770
INFO:root:Epoch[113] Learning rate is 0.0005
100%|██████████| 100/100 [00:01<00:00, 70.64it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[113] Elapsed time 1.417 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.621143
INFO:root:Loading parameters from best epoch (103)
INFO:root:Epoch[114] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 70.08it/s, avg_epoch_loss=6.63]
INFO:root:Epoch[114] Elapsed time 1.428 seconds
INFO:root:Epoch[114] Evaluation metric 'epoch_loss'=6.628909
INFO:root:Epoch[115] Learning rate is 0.00025
10

...

100%|██████████| 100/100 [00:01<00:00, 69.83it/s, avg_epoch_loss=6.87]
INFO:root:Epoch[74] Elapsed time 1.434 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.868439
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.02it/s, avg_epoch_loss=6.75]
INFO:root:Epoch[75] Elapsed time 1.430 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.752930
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.00it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[76] Elapsed time 1.430 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.713698
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 70.17it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[77] Elapsed time 1.427 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.714881
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 100/100 [00:01<00:00, 71.15it/s, avg_epoch_loss=6.87]
INFO:root:Epoch[78] Elapsed time 1.407 second

INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.605070
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 70.45it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[111] Elapsed time 1.421 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.739406
INFO:root:Epoch[112] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 70.20it/s, avg_epoch_loss=6.67]
INFO:root:Epoch[112] Elapsed time 1.426 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.670887
INFO:root:Epoch[113] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 69.02it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[113] Elapsed time 1.450 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.614096
INFO:root:Epoch[114] Learning rate is 0.00025
100%|██████████| 100/100 [00:01<00:00, 70.11it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[114] Elapsed time 1.428 seconds
INFO:root:Epoch[114] Evaluation metric 'epoch_loss'=6.647124
INFO:root:Epoch[115] Learning rate 

INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=6.733170
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 71.18it/s, avg_epoch_loss=6.64]
INFO:root:Epoch[183] Elapsed time 1.406 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.640573
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.87it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[184] Elapsed time 1.433 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.574331
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 69.55it/s, avg_epoch_loss=6.64]
INFO:root:Epoch[185] Elapsed time 1.439 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=6.641203
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 100/100 [00:01<00:00, 70.32it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[186] Elapsed time 1.424 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=6.707347
INFO:root:Epoch[187] Learning rate is 5e-05

       MASE   sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  seed  \
0  1.378992  0.1081  12.262288            0.116796            0.061167    51   

   epochs  num_batches                train data                 eval data  
0     200          100  m4_quarterly_demographic  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.424071,0.107941,13.383298,0.117750,0.063636,42,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.402655,0.109147,16.790780,0.119638,0.073576,43,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.412072,0.110131,15.526972,0.118914,0.069350,44,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.381574,0.108304,14.345824,0.116479,0.061191,45,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.598590,0.114479,16.380390,0.123345,0.072325,46,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.406126,0.108993,13.375444,0.119164,0.063363,47,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.467968,0.110084,13.169481,0.121559,0.064514,48,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.446043,0.112624,16.973728,0.124500,0.072312,49,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.440216,0.111609,18.365581,0.122381,0.076036,50,200,100,m4_quarterly_demographic,m4_quarterly_demographic
0,1.378992,0.108100,12.262288,0.116796,0.061167,51,200,100,m4_quarterly_demographic,m4_quarterly_demographic


In [38]:
results = pd.DataFrame()

for i in range(42,45):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_atm", dataname_b="m4_quarterly_demographic", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_atm.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 200/200 [00:03<00:00, 64.01it/s, avg_epoch_loss=7.69]
INFO:root:Epoch[0] Elapsed time 3.126 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.687089
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 62.92it/s, avg_epoch_loss=7.18]
INFO:root:Epoch[1] Elapsed time 3.180 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.180051
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 64.22it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[2] Elapsed time 3.115 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.193969
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 62.84it/s, avg_epoch_loss=7.1]
INFO:root:Epoch[3] Elapsed time 3.184 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.104381
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 64.10it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[4] Elapsed time 3.121 seconds
INFO:root:Ep

INFO:root:Epoch[37] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.72it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[37] Elapsed time 2.830 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.827728
INFO:root:Epoch[38] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.45it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[38] Elapsed time 2.840 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.792642
INFO:root:Epoch[39] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.97it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[39] Elapsed time 2.819 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.802193
INFO:root:Epoch[40] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 71.72it/s, avg_epoch_loss=6.89]
INFO:root:Epoch[40] Elapsed time 2.790 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.888149
INFO:root:Epoch[41] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.28it/s, avg_epoch_loss

INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.38it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[110] Elapsed time 2.843 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.781739
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.43it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[111] Elapsed time 2.841 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.792256
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.08it/s, avg_epoch_loss=6.76]
INFO:root:Epoch[112] Elapsed time 2.897 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.756591
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.71it/s, avg_epoch_loss=6.67]
INFO:root:Epoch[113] Elapsed time 2.870 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.674271
INFO:root:Epoch[114] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.07it/s, avg_epoch_

...

INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=6.432178
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.10it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[220] Elapsed time 2.854 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=6.491991
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.76it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[221] Elapsed time 2.869 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=6.452930
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.59it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[222] Elapsed time 2.875 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=6.335701
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.42it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[223] Elapsed time 2.841 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=6.459334
INFO:root:Epoch[224] Learning rate is 5e-05

INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.92it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[293] Elapsed time 2.782 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.307912
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 72.67it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[294] Elapsed time 2.754 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.421142
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.79it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[295] Elapsed time 2.787 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.470115
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.56it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[296] Elapsed time 2.796 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.417148
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.50it/s, avg_epoch_

INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=6.490694
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.81it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[330] Elapsed time 2.787 seconds
INFO:root:Epoch[330] Evaluation metric 'epoch_loss'=6.366264
INFO:root:Epoch[331] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.65it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[331] Elapsed time 2.873 seconds
INFO:root:Epoch[331] Evaluation metric 'epoch_loss'=6.442432
INFO:root:Epoch[332] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.33it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[332] Elapsed time 2.805 seconds
INFO:root:Epoch[332] Evaluation metric 'epoch_loss'=6.395636
INFO:root:Epoch[333] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.31it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[333] Elapsed time 2.806 seconds
INFO:root:Epoch[333] Evaluation metric 'epoch_loss'=6.392166
INFO:root:Epoch[334] Learning rate is 5e-05


       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.409613  0.107282  12.675523            0.116014            0.063689   

   seed  epochs  num_batches        train data                 eval data  
0    44     400          200  m4_quarterly_atm  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.388286,0.108630,11.474272,0.117209,0.067408,42,400,200,m4_quarterly_atm,m4_quarterly_demographic
0,1.483827,0.111884,15.030332,0.118564,0.065811,43,400,200,m4_quarterly_atm,m4_quarterly_demographic
0,1.409613,0.107282,12.675523,0.116014,0.063689,44,400,200,m4_quarterly_atm,m4_quarterly_demographic


In [39]:
results = pd.DataFrame()

for i in range(42,45):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_demographic", dataname_b="m4_quarterly_demographic", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 42


100%|██████████| 200/200 [00:02<00:00, 68.36it/s, avg_epoch_loss=7.66]
INFO:root:Epoch[0] Elapsed time 2.927 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.656172
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 69.47it/s, avg_epoch_loss=7.32]
INFO:root:Epoch[1] Elapsed time 2.880 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.319610
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.08it/s, avg_epoch_loss=7.17]
INFO:root:Epoch[2] Elapsed time 2.855 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.165541
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.85it/s, avg_epoch_loss=7.15]
INFO:root:Epoch[3] Elapsed time 2.824 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.154277
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 71.65it/s, avg_epoch_loss=7.1]
INFO:root:Epoch[4] Elapsed time 2.793 seconds
INFO:root:Ep

INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.45it/s, avg_epoch_loss=6.53]
INFO:root:Epoch[74] Elapsed time 2.840 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.526512
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 69.91it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[75] Elapsed time 2.862 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.623791
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.07it/s, avg_epoch_loss=6.56]
INFO:root:Epoch[76] Elapsed time 2.856 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.564563
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.61it/s, avg_epoch_loss=6.6]
INFO:root:Epoch[77] Elapsed time 2.834 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.603682
INFO:root:Epoch[78] Learning rate is 0.00025
100%|██████████| 200/200 [00:02<00:00, 70.31it/s, avg_epoch_loss

100%|██████████| 200/200 [00:02<00:00, 71.01it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[110] Elapsed time 2.818 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.612082
INFO:root:Epoch[111] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:02<00:00, 71.89it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[111] Elapsed time 2.783 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.523712
INFO:root:Epoch[112] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:02<00:00, 72.55it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[112] Elapsed time 2.758 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.610216
INFO:root:Epoch[113] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:02<00:00, 71.76it/s, avg_epoch_loss=6.58]
INFO:root:Epoch[113] Elapsed time 2.788 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=6.578775
INFO:root:Epoch[114] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:02<00:00, 71.75it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[114] 

...

INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.82it/s, avg_epoch_loss=6.6]
INFO:root:Epoch[293] Elapsed time 2.825 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.597501
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.59it/s, avg_epoch_loss=6.48]
INFO:root:Epoch[294] Elapsed time 2.876 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.476171
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.54it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[295] Elapsed time 2.837 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.618753
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 71.69it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[296] Elapsed time 2.791 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.486521
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.38it/s, avg_epoch_l

INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=6.510249
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.86it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[330] Elapsed time 2.865 seconds
INFO:root:Epoch[330] Evaluation metric 'epoch_loss'=6.547829
INFO:root:Epoch[331] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.36it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[331] Elapsed time 2.844 seconds
INFO:root:Epoch[331] Evaluation metric 'epoch_loss'=6.519952
INFO:root:Epoch[332] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 70.30it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[332] Elapsed time 2.846 seconds
INFO:root:Epoch[332] Evaluation metric 'epoch_loss'=6.499357
INFO:root:Epoch[333] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 69.77it/s, avg_epoch_loss=6.59]
INFO:root:Epoch[333] Elapsed time 2.868 seconds
INFO:root:Epoch[333] Evaluation metric 'epoch_loss'=6.589871
INFO:root:Epoch[334] Learning rate is 5e-05


       MASE    sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  seed  \
0  1.347529  0.10691  12.337855            0.115888            0.062129    42   

   epochs  num_batches                train data                 eval data  
0     400          200  m4_quarterly_demographic  m4_quarterly_demographic  
Next seed: 43


100%|██████████| 200/200 [00:02<00:00, 69.27it/s, avg_epoch_loss=7.59]
INFO:root:Epoch[0] Elapsed time 2.889 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.585795
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:02<00:00, 70.88it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[1] Elapsed time 2.823 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.273609
INFO:root:Epoch[2] Learning rate is 0.001
  0%|          | 0/200 [00:01<?, ?it/s, avg_epoch_loss=7.19]


KeyboardInterrupt: 

In [40]:
results

,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.347529,0.10691,12.337855,0.115888,0.062129,42,400,200,m4_quarterly_demographic,m4_quarterly_demographic


In [16]:
results = pd.DataFrame()

for i in range(43,45):
    print("Next seed:", i)
    df = deepar_train_eval(dataname_a="m4_quarterly_demographic", dataname_b="m4_quarterly_demographic", seed=i, epochs=400, num_batches=200)
    pprint(df)
    results = results.append(df)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly_demographic.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 22684


Next seed: 43


100%|██████████| 200/200 [00:04<00:00, 48.39it/s, avg_epoch_loss=7.59]
INFO:root:Epoch[0] Elapsed time 4.135 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.585795
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 50.25it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[1] Elapsed time 3.982 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.273609
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 50.00it/s, avg_epoch_loss=7.18]
INFO:root:Epoch[2] Elapsed time 4.002 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.184588
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 49.21it/s, avg_epoch_loss=7.15]
INFO:root:Epoch[3] Elapsed time 4.066 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.148316
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 53.33it/s, avg_epoch_loss=7.12]
INFO:root:Epoch[4] Elapsed time 3.752 seconds
INFO:root:E

100%|██████████| 200/200 [00:04<00:00, 48.67it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[37] Elapsed time 4.112 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.939980
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 51.03it/s, avg_epoch_loss=6.84]
INFO:root:Epoch[38] Elapsed time 3.921 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.836388
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 47.59it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[39] Elapsed time 4.204 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.900490
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 49.91it/s, avg_epoch_loss=6.88]
INFO:root:Epoch[40] Elapsed time 4.009 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.876343
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 49.95it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[41] Elapsed time 4.007 seconds

100%|██████████| 200/200 [00:03<00:00, 50.34it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[74] Elapsed time 3.975 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.606843
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 47.19it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[75] Elapsed time 4.241 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.606713
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 48.59it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[76] Elapsed time 4.118 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.645625
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 200/200 [00:03<00:00, 51.25it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[77] Elapsed time 3.904 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=6.573122
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 46.70it/s, avg_epoch_loss=6.67]
INFO:root:Epoch[78] Elapsed time 4.285 second

...

INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=6.441210
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 61.62it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[293] Elapsed time 3.247 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.490632
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 62.53it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[294] Elapsed time 3.200 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.421573
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 68.39it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[295] Elapsed time 2.926 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.524650
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 66.13it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[296] Elapsed time 3.026 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.422789
INFO:root:Epoch[297] Learning rate is 5e-05

100%|██████████| 200/200 [00:03<00:00, 60.11it/s, avg_epoch_loss=6.48]
INFO:root:Epoch[329] Elapsed time 3.329 seconds
INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=6.479032
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 59.41it/s, avg_epoch_loss=6.49]
INFO:root:Epoch[330] Elapsed time 3.368 seconds
INFO:root:Epoch[330] Evaluation metric 'epoch_loss'=6.485396
INFO:root:Epoch[331] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 64.04it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[331] Elapsed time 3.125 seconds
INFO:root:Epoch[331] Evaluation metric 'epoch_loss'=6.418291
INFO:root:Epoch[332] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 62.88it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[332] Elapsed time 3.182 seconds
INFO:root:Epoch[332] Evaluation metric 'epoch_loss'=6.501887
INFO:root:Epoch[333] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 65.75it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[333] Elapsed time 

INFO:root:Epoch[366] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 64.45it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[366] Elapsed time 3.105 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=6.401947
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 63.34it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[367] Elapsed time 3.160 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=6.500229
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 200/200 [00:02<00:00, 68.82it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[368] Elapsed time 2.908 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=6.371883
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 66.12it/s, avg_epoch_loss=6.51]
INFO:root:Epoch[369] Elapsed time 3.026 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=6.510040
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 200/200 [00:03<00:00, 57.25it/s, avg_epoch_lo

       MASE     sMAPE       MSIS  wQuantileLoss[0.5]  wQuantileLoss[0.9]  \
0  1.333239  0.105234  11.808035            0.113221            0.060194   

   seed  epochs  num_batches                train data  \
0    44     400          200  m4_quarterly_demographic   

                  eval data  
0  m4_quarterly_demographic  


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],seed,epochs,num_batches,train data,eval data
0,1.314023,0.104997,11.752810,0.114577,0.060521,43,400,200,m4_quarterly_demographic,m4_quarterly_demographic
0,1.333239,0.105234,11.808035,0.113221,0.060194,44,400,200,m4_quarterly_demographic,m4_quarterly_demographic
